## Datenmodell-Schema DIGiHAST

### Konzept

Erstellung eines skalierbaren Datenmodells zur detaillierten Beschreibung eines Fernwärmenetzes mit vernetzten Hausstationen (iHAST) sowie Netzknoten (iKNOTEN).

### Import dependency

In [15]:
import json
from pydantic import ConfigDict, BaseModel
from pydantic.fields import Field, FieldInfo
from typing import Any, Optional
from filip.models import FiwareHeader
from filip.models.ngsi_v2.context import ContextEntityKeyValues
from filip.clients.ngsi_v2.cb import ContextBrokerClient

### Erstellung der Datenmodelle

#### Positionsdaten

In [9]:
class PostalAddress(BaseModel):
    """ 
    Model for address (maybe adapt to https://schema.org/PostalAddress)
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)
    
    streetName: str = Field(
        description="Street name",
    )
    streetNumber: str = Field(
        description="Street number",
    )
    city: str = Field(
        description="City",
    )
    postalCode: int = Field(
        description="Postal code",
    )
    

class Location(BaseModel):
    """
    Model for location
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)
    
    latitude: float = Field(
        description="Latitude",
    )
    longitude: float = Field(
        description="Longitude",
    )

#### Datenmodell WMZ

In [5]:
class BJ(BaseModel):
    """
    Model for a heat meter
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)  

    fwtemp: float = Field(
        default=None,
        description="dh supply temperature in degree Celsius"
    )
    rttemp: float = Field(
        default=None,
        description="dh return temperature in degree Celsius"
    )
    flow: float = Field(
        default=None,
        description="dh flow in kubikmeter per hour"
    )
    power: float = Field(
        default=None,
        description="dh power in kilowatt"
    )
    volume: float = Field(
        default=None,
        description="dh volume in kubikmeter"
    )
    energy: float = Field(
        default=None,
        description="dh energy in kilowatthour"
    )
    ref_entity: Any = Field(
        default=None,
        description="Reference to the entity"
    )

#### Datenmodell Sensoren

In [26]:
class BT(BaseModel):
    """
    Model for a temperature sensor
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)

    temperature: float = Field(
        default=None,
        description="measure temperature in degree Celsius"
    )
    ref_entity: Any = Field(
        default=None,
        description="Reference to the entity"
    )

class BP(BaseModel):
    """
    Model for a pressure sensor
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)
    
    pressure: float = Field(
        default=None,
        description="measure pressure in bar"
    )
    ref_entity: Any = Field(
        default=None,
        description="Reference to the entity"
    )
    
class BF(BaseModel):
    """
    Model for a flow sensor
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)

    flow: float = Field(
        default=None,
        description="measure flow in kubikmeter per hour"
    )
    ref_entity: Any = Field(
        default=None,
        description="Reference to the entity"
    )

class BS(BaseModel):
    """
    Model for a rotatiol frequency
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)

    rotational_frequency: float = Field(
        default=None,
        description="measure rotatiol frequency in rpm"
    )
    ref_entity: Any = Field(
        default=None,
        description="Reference to the entity"
    )
    

#### Datenmodell für iHAST

In [27]:
class iHAST(BaseModel):
    """
    Model for a substation
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)

    address: Optional[PostalAddress]
    location: Optional[Location]
    subnet: Optional[str] = Field(
        alias="subnet",
        default=None,
        description="subnet "
    )
    connected_load: Optional[float]= Field(
        alias="connectedLoad",
        default=None,
        description="Connected load of a substation"
    )
    design_flow: Optional[float]= Field(
        alias="design_flow",
        default=None,
        description="Design flow of a substation"
    )
    fwtemp_tcr: Optional[float]= Field(
        alias="fwtemp_tcr",
        default=None,
        description="Technical connection regulations for supply temperature"
    )
    rttemp_tcr: Optional[float]= Field(
        alias="rttemp_tcr",
        default=None,
        description="Technical connection regulations for return temperature"
    )

#### Datenmodell für iKNOTEN

In [28]:
class iKNOTEN(BaseModel):
    """
    Model for a substation
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)

    location: Optional[Location]
    subnet: Optional[str] = Field(
        alias="subnet",
        default=None,
        description="subnet "
    )

### Umsetzung als FIWARE-Modell

In [ ]:
class BJFIWARE(BJ, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="BJ"
        description="Type of the heat meter"
    )
    
class BTFIWARE(BT, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="BT"
        description="Type of the temperature sensor"
    )
    
class BPFIWARE(BP, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="BP",
        description="Type of the pressure sensor"
    )
    
class BFFIWARE(BF, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="BF",
        description="Type of the flow sensor"
    )
    
class BSFIWARE(BS, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="BS",
        description="Type of the rotational frequency sensor"
    )
    
class iHASTFIWARE(iHAST, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="iHAST",
        description="Type of the substation"
    )
    
class iKNOTENFIWARE(iKNOTEN, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="iKNOTEN",
        description="Type of the substation"
    )

### Nutzung mit FIWARE-Instanz

#### Initialisieren des FIWARE-API-Clients

In [ ]:
CB_URL = "http://localhost:1026"
SERVICE = 'digihast_datamodel_test'
SERVICE_PATH = '/'
fiware_header = FiwareHeader(
    service=SERVICE,
    service_path=SERVICE_PATH
)
cb_client = ContextBrokerClient(
    url=CB_URL,
    fiware_header=fiware_header
)

#### Anlegen von Datenpunkten

In [ ]:
entities = []

# iHAST
entities.append(iHASTFIWARE(
    id="iHAST:1"
))

# Heat meter BJ
entities.append(BJFIWARE(
    id="BJ:1", 
    isPartOf=entities[-1].id
))

for entity in entities:
    cb_client.post_entity(entity, key_values=True, update=True)

### Export des Datenmodells

In [29]:
file_path = "./schemes/"

bj_json_schema = BJ.schema()
with open(f"{file_path}/bj_schema.json", "w") as f:
    json.dump(bj_json_schema, f, indent=4)

bt_json_schema = BT.schema()
with open(f"{file_path}/bt_schema.json", "w") as f:
    json.dump(bt_json_schema, f, indent=4)

bp_json_schema = BP.schema()
with open(f"{file_path}/bp_schema.json", "w") as f:
    json.dump(bp_json_schema, f, indent=4)

bf_json_schema = BF.schema()
with open(f"{file_path}/bf_schema.json", "w") as f:
    json.dump(bf_json_schema, f, indent=4)

bs_json_schema = BS.schema()
with open(f"{file_path}/bs_schema.json", "w") as f:
    json.dump(bs_json_schema, f, indent=4)

ihast_json_schema = iHAST.schema()
with open(f"{file_path}/ihast_schema.json", "w") as f:
    json.dump(ihast_json_schema, f, indent=4)

iknoten_json_schema = iKNOTEN.schema()
with open(f"{file_path}/iknoten_schema.json", "w") as f:
    json.dump(iknoten_json_schema, f, indent=4)